# Predicting Sea Surface Temperatures

## Set up

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Reading the data
sea_temp = pd.read_csv("/content/Average_Sea_Surface_Temps.csv")
sea_variables = pd.read_csv("/content/Avg_Variables.csv")

In [3]:
# Data overview
sea_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345646 entries, 0 to 345645
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   month                 345646 non-null  int64  
 1   day                   345646 non-null  int64  
 2   timestamp             345646 non-null  object 
 3   avg_sea_surface_temp  345646 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 10.5+ MB


In [4]:
# Drop missing row
sea_temp.dropna(inplace = True)

In [5]:
# Change to datetime
sea_temp['timestamp'] = pd.to_datetime(sea_temp['timestamp'])

In [6]:
# Check summary statistics
sea_temp.describe()

,month,day,avg_sea_surface_temp
count,345646.000000,345646.000000,345646.000000
mean,6.502367,15.718996,20.237942
std,3.442440,8.796251,4.103446
min,1.000000,1.000000,0.200000
25%,4.000000,8.000000,17.409664
50%,7.000000,16.000000,20.675676
75%,9.000000,23.000000,23.308325
max,12.000000,31.000000,33.000000


In [7]:
# Scaling the data

from sklearn.preprocessing import MinMaxScaler

X = sea_temp[['avg_sea_surface_temp']]

scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(X)

In [8]:
# Splitting the data
split_ratio = 0.8
train_size = int(len(sea_temp) * split_ratio)

# Single Variable MLP

## Model 1

lr = 0.0001
momentum= 0.9
epochs = 500
sequence_length = 49

In [ ]:
# Creating sequences
sequence_length = 49

def create_sequences(data, sequence_length):

  total_sequence=[]
  prices = []

  for day in range(0, (len(data) - sequence_length)):
    start_index = day
    end_index = sequence_length + day
    day_sequence = data[start_index:end_index]
    day_price = data[end_index]

    total_sequence.append(day_sequence)
    prices.append(day_price)
  return np.array(total_sequence), np.array(prices)

sequences = create_sequences(scaled_data, sequence_length)
X = sequences[0]
y = sequences[1]

In [ ]:
import torch

# Split the data into training and testing sets
train_size = int(X.shape[0]* split_ratio)
X_train = torch.tensor(X[:train_size]).float()
y_train = torch.tensor(y[:train_size]).float()
X_test = torch.tensor(X[train_size:]).float()
y_test = torch.tensor(y[train_size:]).float()

In [ ]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
      super(MLP, self).__init__()
      self.model = nn.Sequential(
          nn.Linear(input_size, 20),
          nn.Linear(20, 10),
          nn.Linear(10, 1)
      )

    def forward(self, x):
      x = x.view(-1, input_size)
      x = self.model(x)
      return x

# Instantiate the model
input_size = sequence_length
model = MLP(input_size)

In [ ]:
# Model 1 Parameters

lr = 0.0001
momentum= 0.9
epochs = 500

In [ ]:
import torch.optim as optim

criterion_training = nn.L1Loss()
criterion_testing = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
# Putting the y data in the correct format

y_test_actual = torch.tensor(scaler.inverse_transform(y_test))
y_train_actual = torch.tensor(scaler.inverse_transform(y_train))

# Initializing a best model for later use
best_test_error = torch.tensor(float(500))
best_model_state = None

# Train the model
for epoch in range(epochs):

    optimizer.zero_grad()
    model.train()
    pred = model(X_train)

    predictions_train_actual = torch.tensor(scaler.inverse_transform(pred.detach()))
    actual_train_error = criterion_training(predictions_train_actual, y_train_actual)

    loss = criterion_training(pred, y_train)
    # Update model here based on error
    loss.backward()
    optimizer.step()

    model.eval()
    # Evaluate the model on the test data
    with torch.no_grad():
      # Evaluate the model here.
      pred_test = model(X_test)
      loss_test = criterion_testing(pred_test, y_test)

      predictions_test_actual = torch.tensor(scaler.inverse_transform(pred_test.detach()))
      actual_test_error = criterion_testing(predictions_test_actual, y_test_actual)

    #defining a best model
    if actual_test_error < best_test_error:
        best_test_error = actual_test_error
        best_model_state = model.state_dict()

    if epoch %10 == 0:
      print(f"Epoch {epoch}: Training Actual Error= {actual_train_error}, Test Actual Error= {actual_test_error}")

Epoch 0: Training Actual Error= 22.45330124147956, Test Actual Error= 21.19702731447268
Epoch 10: Training Actual Error= 21.831552531370942, Test Actual Error= 20.510239049424204
Epoch 20: Training Actual Error= 20.648876386816163, Test Actual Error= 19.337201653328183
Epoch 30: Training Actual Error= 19.288882586416268, Test Actual Error= 18.011851005306102
Epoch 40: Training Actual Error= 17.88191932958824, Test Actual Error= 16.64706751795244
Epoch 50: Training Actual Error= 16.468716272339222, Test Actual Error= 15.277708502744442
Epoch 60: Training Actual Error= 15.058684140565664, Test Actual Error= 13.911513924070514
Epoch 70: Training Actual Error= 13.650717392203491, Test Actual Error= 12.546725514618144
Epoch 80: Training Actual Error= 12.240749420632506, Test Actual Error= 11.178649491471392
Epoch 90: Training Actual Error= 10.829766122469158, Test Actual Error= 9.803452842610076
Epoch 100: Training Actual Error= 9.45922403468723, Test Actual Error= 8.433363402196361
Epoch 1

## Model 2

lr = 0.01
momentum= 0.9
epochs = 500
sequence_length = 49

In [ ]:
# Creating sequences
sequence_length = 49

def create_sequences(data, sequence_length):

  total_sequence=[]
  prices = []

  for day in range(0, (len(data) - sequence_length)):
    start_index = day
    end_index = sequence_length + day
    day_sequence = data[start_index:end_index]
    day_price = data[end_index]

    total_sequence.append(day_sequence)
    prices.append(day_price)
  return np.array(total_sequence), np.array(prices)

sequences = create_sequences(scaled_data, sequence_length)
X = sequences[0]
y = sequences[1]

In [ ]:
import torch

# Split the data into training and testing sets
train_size = int(X.shape[0]* split_ratio)
X_train = torch.tensor(X[:train_size]).float()
y_train = torch.tensor(y[:train_size]).float()
X_test = torch.tensor(X[train_size:]).float()
y_test = torch.tensor(y[train_size:]).float()

In [ ]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
      super(MLP, self).__init__()
      self.model = nn.Sequential(
          nn.Linear(input_size, 20),
          nn.Linear(20, 10),
          nn.Linear(10, 1)
      )

    def forward(self, x):
      x = x.view(-1, input_size)
      x = self.model(x)
      return x

# Instantiate the model
input_size = sequence_length
model = MLP(input_size)

In [ ]:
# Model 2 Parameters

lr = 0.01
momentum= 0.9
epochs = 500

In [ ]:
import torch.optim as optim

criterion_training = nn.L1Loss()
criterion_testing = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
# Putting the y data in the correct format

y_test_actual = torch.tensor(scaler.inverse_transform(y_test))
y_train_actual = torch.tensor(scaler.inverse_transform(y_train))

# Initializing a best model for later use
best_test_error = torch.tensor(float(500))
best_model_state = None

# Train the model
for epoch in range(epochs):

    optimizer.zero_grad()
    model.train()
    pred = model(X_train)

    predictions_train_actual = torch.tensor(scaler.inverse_transform(pred.detach()))
    actual_train_error = criterion_training(predictions_train_actual, y_train_actual)

    loss = criterion_training(pred, y_train)
    # Update model here based on error
    loss.backward()
    optimizer.step()

    model.eval()
    # Evaluate the model on the test data
    with torch.no_grad():
      # Evaluate the model here.
      pred_test = model(X_test)
      loss_test = criterion_testing(pred_test, y_test)

      predictions_test_actual = torch.tensor(scaler.inverse_transform(pred_test.detach()))
      actual_test_error = criterion_testing(predictions_test_actual, y_test_actual)

    #defining a best model
    if actual_test_error < best_test_error:
        best_test_error = actual_test_error
        best_model_state = model.state_dict()

    if epoch %10 == 0:
      print(f"Epoch {epoch}: Training Actual Error= {actual_train_error}, Test Actual Error= {actual_test_error}")

Epoch 0: Training Actual Error= 17.53795744536497, Test Actual Error= 14.405030491332678
Epoch 10: Training Actual Error= 4.811422331832247, Test Actual Error= 5.616880186946321
Epoch 20: Training Actual Error= 3.4549581636838567, Test Actual Error= 2.9233915666357126
Epoch 30: Training Actual Error= 3.620495761442022, Test Actual Error= 3.7498279578491385
Epoch 40: Training Actual Error= 3.1088762167873276, Test Actual Error= 3.333050749512061
Epoch 50: Training Actual Error= 3.1084706942287523, Test Actual Error= 2.941747689385536
Epoch 60: Training Actual Error= 3.063041389141574, Test Actual Error= 2.886245172629219
Epoch 70: Training Actual Error= 3.017299736776088, Test Actual Error= 2.931553160899784
Epoch 80: Training Actual Error= 2.988841576219572, Test Actual Error= 2.9560964075314673
Epoch 90: Training Actual Error= 2.9564067539131944, Test Actual Error= 2.932826115724682
Epoch 100: Training Actual Error= 2.9232118519457297, Test Actual Error= 2.892908353959124
Epoch 110: T

## Model 3

lr = 0.01
momentum= 0.9
epochs = 500
sequence_length = 100

In [ ]:
# Creating sequences
sequence_length = 100

def create_sequences(data, sequence_length):

  total_sequence=[]
  prices = []

  for day in range(0, (len(data) - sequence_length)):
    start_index = day
    end_index = sequence_length + day
    day_sequence = data[start_index:end_index]
    day_price = data[end_index]

    total_sequence.append(day_sequence)
    prices.append(day_price)
  return np.array(total_sequence), np.array(prices)

sequences = create_sequences(scaled_data, sequence_length)
X = sequences[0]
y = sequences[1]

In [ ]:
import torch

# Split the data into training and testing sets
train_size = int(X.shape[0]* split_ratio)
X_train = torch.tensor(X[:train_size]).float()
y_train = torch.tensor(y[:train_size]).float()
X_test = torch.tensor(X[train_size:]).float()
y_test = torch.tensor(y[train_size:]).float()

In [ ]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
      super(MLP, self).__init__()
      self.model = nn.Sequential(
          nn.Linear(input_size, 20),
          nn.Linear(20, 10),
          nn.Linear(10, 1)
      )

    def forward(self, x):
      x = x.view(-1, input_size)
      x = self.model(x)
      return x

# Instantiate the model
input_size = sequence_length
model = MLP(input_size)

In [ ]:
# Model 3 Parameters

lr = 0.01
momentum= 0.9
epochs = 500

In [ ]:
import torch.optim as optim

criterion_training = nn.L1Loss()
criterion_testing = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
# Putting the y data in the correct format

y_test_actual = torch.tensor(scaler.inverse_transform(y_test))
y_train_actual = torch.tensor(scaler.inverse_transform(y_train))

# Initializing a best model for later use
best_test_error = torch.tensor(float(500))
best_model_state = None

# Train the model
for epoch in range(epochs):

    optimizer.zero_grad()
    model.train()
    pred = model(X_train)

    predictions_train_actual = torch.tensor(scaler.inverse_transform(pred.detach()))
    actual_train_error = criterion_training(predictions_train_actual, y_train_actual)

    loss = criterion_training(pred, y_train)
    # Update model here based on error
    loss.backward()
    optimizer.step()

    model.eval()
    # Evaluate the model on the test data
    with torch.no_grad():
      # Evaluate the model here.
      pred_test = model(X_test)
      loss_test = criterion_testing(pred_test, y_test)

      predictions_test_actual = torch.tensor(scaler.inverse_transform(pred_test.detach()))
      actual_test_error = criterion_testing(predictions_test_actual, y_test_actual)

    #defining a best model
    if actual_test_error < best_test_error:
        best_test_error = actual_test_error
        best_model_state = model.state_dict()

    if epoch %10 == 0:
      print(f"Epoch {epoch}: Training Actual Error= {actual_train_error}, Test Actual Error= {actual_test_error}")

Epoch 0: Training Actual Error= 7.462508916412034, Test Actual Error= 5.428122023448139
Epoch 10: Training Actual Error= 3.7325993992950175, Test Actual Error= 3.224638116250437
Epoch 20: Training Actual Error= 3.2160645525342955, Test Actual Error= 3.0916808997818013
Epoch 30: Training Actual Error= 3.226903514645619, Test Actual Error= 3.545504198002325
Epoch 40: Training Actual Error= 3.086976513391635, Test Actual Error= 3.113865198747257
Epoch 50: Training Actual Error= 3.0744560169641026, Test Actual Error= 2.9666427670618227
Epoch 60: Training Actual Error= 3.026772879807613, Test Actual Error= 3.005127954886405
Epoch 70: Training Actual Error= 2.9968866525642497, Test Actual Error= 3.039240457755735
Epoch 80: Training Actual Error= 2.9625745331080386, Test Actual Error= 3.0195279496966827
Epoch 90: Training Actual Error= 2.9265157010906893, Test Actual Error= 2.981616787630075
Epoch 100: Training Actual Error= 2.88922595903435, Test Actual Error= 2.9451509850806734
Epoch 110: T

So far this is the best model

## Model 4

lr = 0.01
momentum= 0.9
epochs = 500
sequence_length = 100

layers changed:
          nn.Linear(input_size, 40),
          nn.Linear(40, 20),
          nn.Linear(20, 1)

In [ ]:
# Creating sequences
sequence_length = 100

def create_sequences(data, sequence_length):

  total_sequence=[]
  prices = []

  for day in range(0, (len(data) - sequence_length)):
    start_index = day
    end_index = sequence_length + day
    day_sequence = data[start_index:end_index]
    day_price = data[end_index]

    total_sequence.append(day_sequence)
    prices.append(day_price)
  return np.array(total_sequence), np.array(prices)

sequences = create_sequences(scaled_data, sequence_length)
X = sequences[0]
y = sequences[1]

In [ ]:
import torch

# Split the data into training and testing sets
train_size = int(X.shape[0]* split_ratio)
X_train = torch.tensor(X[:train_size]).float()
y_train = torch.tensor(y[:train_size]).float()
X_test = torch.tensor(X[train_size:]).float()
y_test = torch.tensor(y[train_size:]).float()

In [ ]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
      super(MLP, self).__init__()
      self.model = nn.Sequential(
          nn.Linear(input_size, 40),
          nn.Linear(40, 20),
          nn.Linear(20, 1)
      )

    def forward(self, x):
      x = x.view(-1, input_size)
      x = self.model(x)
      return x

# Instantiate the model
input_size = sequence_length
model = MLP(input_size)

In [ ]:
# Model 4 Parameters

lr = 0.01
momentum= 0.9
epochs = 500

In [ ]:
import torch.optim as optim

criterion_training = nn.L1Loss()
criterion_testing = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
# Putting the y data in the correct format

y_test_actual = torch.tensor(scaler.inverse_transform(y_test))
y_train_actual = torch.tensor(scaler.inverse_transform(y_train))

# Initializing a best model for later use
best_test_error = torch.tensor(float(500))
best_model_state = None

# Train the model
for epoch in range(epochs):

    optimizer.zero_grad()
    model.train()
    pred = model(X_train)

    predictions_train_actual = torch.tensor(scaler.inverse_transform(pred.detach()))
    actual_train_error = criterion_training(predictions_train_actual, y_train_actual)

    loss = criterion_training(pred, y_train)
    # Update model here based on error
    loss.backward()
    optimizer.step()

    model.eval()
    # Evaluate the model on the test data
    with torch.no_grad():
      # Evaluate the model here.
      pred_test = model(X_test)
      loss_test = criterion_testing(pred_test, y_test)

      predictions_test_actual = torch.tensor(scaler.inverse_transform(pred_test.detach()))
      actual_test_error = criterion_testing(predictions_test_actual, y_test_actual)

    #defining a best model
    if actual_test_error < best_test_error:
        best_test_error = actual_test_error
        best_model_state = model.state_dict()

    if epoch %10 == 0:
      print(f"Epoch {epoch}: Training Actual Error= {actual_train_error}, Test Actual Error= {actual_test_error}")

Epoch 0: Training Actual Error= 24.136541951597682, Test Actual Error= 19.23670418343941
Epoch 10: Training Actual Error= 10.268979494118796, Test Actual Error= 7.861134949079852
Epoch 20: Training Actual Error= 4.8569921345065605, Test Actual Error= 2.94252055123416
Epoch 30: Training Actual Error= 2.9921973156546464, Test Actual Error= 3.192271655218643
Epoch 40: Training Actual Error= 3.119366621741424, Test Actual Error= 3.2010983003585594
Epoch 50: Training Actual Error= 2.9482338820090797, Test Actual Error= 2.830103993862673
Epoch 60: Training Actual Error= 2.8623569441710055, Test Actual Error= 2.736982449909881
Epoch 70: Training Actual Error= 2.830896736675459, Test Actual Error= 2.746487756552097
Epoch 80: Training Actual Error= 2.7782363329599216, Test Actual Error= 2.778138934260614
Epoch 90: Training Actual Error= 2.7377138329538786, Test Actual Error= 2.7406659001484566
Epoch 100: Training Actual Error= 2.693845817862434, Test Actual Error= 2.6978011359149545
Epoch 110: 

## Model 4.5

lr = 0.01
momentum= 0.9
epochs = 500
sequence_length = 100

layers changed:
          nn.Linear(input_size, 80),
          nn.Linear(80, 60),
          nn.Linear(60, 1)

In [ ]:
# Creating sequences
sequence_length = 100

def create_sequences(data, sequence_length):

  total_sequence=[]
  prices = []

  for day in range(0, (len(data) - sequence_length)):
    start_index = day
    end_index = sequence_length + day
    day_sequence = data[start_index:end_index]
    day_price = data[end_index]

    total_sequence.append(day_sequence)
    prices.append(day_price)
  return np.array(total_sequence), np.array(prices)

sequences = create_sequences(scaled_data, sequence_length)
X = sequences[0]
y = sequences[1]

In [ ]:
import torch

# Split the data into training and testing sets
train_size = int(X.shape[0]* split_ratio)
X_train = torch.tensor(X[:train_size]).float()
y_train = torch.tensor(y[:train_size]).float()
X_test = torch.tensor(X[train_size:]).float()
y_test = torch.tensor(y[train_size:]).float()

In [ ]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
      super(MLP, self).__init__()
      self.model = nn.Sequential(
          nn.Linear(input_size, 80),
          nn.Linear(80, 60),
          nn.Linear(60, 1)
      )

    def forward(self, x):
      x = x.view(-1, input_size)
      x = self.model(x)
      return x

# Instantiate the model
input_size = sequence_length
model = MLP(input_size)

In [ ]:
# Model 4 Parameters

lr = 0.01
momentum= 0.9
epochs = 500

In [ ]:
import torch.optim as optim

criterion_training = nn.L1Loss()
criterion_testing = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
# Putting the y data in the correct format

y_test_actual = torch.tensor(scaler.inverse_transform(y_test))
y_train_actual = torch.tensor(scaler.inverse_transform(y_train))

# Initializing a best model for later use
best_test_error = torch.tensor(float(500))
best_model_state = None

# Train the model
for epoch in range(epochs):

    optimizer.zero_grad()
    model.train()
    pred = model(X_train)

    predictions_train_actual = torch.tensor(scaler.inverse_transform(pred.detach()))
    actual_train_error = criterion_training(predictions_train_actual, y_train_actual)

    loss = criterion_training(pred, y_train)
    # Update model here based on error
    loss.backward()
    optimizer.step()

    model.eval()
    # Evaluate the model on the test data
    with torch.no_grad():
      # Evaluate the model here.
      pred_test = model(X_test)
      loss_test = criterion_testing(pred_test, y_test)

      predictions_test_actual = torch.tensor(scaler.inverse_transform(pred_test.detach()))
      actual_test_error = criterion_testing(predictions_test_actual, y_test_actual)

    #defining a best model
    if actual_test_error < best_test_error:
        best_test_error = actual_test_error
        best_model_state = model.state_dict()

    if epoch %10 == 0:
      print(f"Epoch {epoch}: Training Actual Error= {actual_train_error}, Test Actual Error= {actual_test_error}")

Epoch 0: Training Actual Error= 12.986954716124965, Test Actual Error= 8.59209168623479
Epoch 10: Training Actual Error= 6.052610706770939, Test Actual Error= 2.802955639332752
Epoch 20: Training Actual Error= 3.353795992081187, Test Actual Error= 2.969491170551183
Epoch 30: Training Actual Error= 2.937998686767853, Test Actual Error= 3.19467274897567
Epoch 40: Training Actual Error= 2.902318305521516, Test Actual Error= 3.1927043653770384
Epoch 50: Training Actual Error= 2.8722794803072884, Test Actual Error= 3.0607607832374626
Epoch 60: Training Actual Error= 2.810818730752756, Test Actual Error= 2.9157963080017906
Epoch 70: Training Actual Error= 2.735806522829881, Test Actual Error= 2.789161533007848
Epoch 80: Training Actual Error= 2.6719044607537366, Test Actual Error= 2.715984067739634
Epoch 90: Training Actual Error= 2.6118737578490054, Test Actual Error= 2.7139232294561375
Epoch 100: Training Actual Error= 2.556043719337751, Test Actual Error= 2.6741819768131525
Epoch 110: Tra

## Model 5

lr = 0.01
momentum= 0.9
epochs = 500
sequence_length = 100

layers changed:
          nn.Linear(input_size, 20),
          nn.Linear(20, 10),
          nn.Linear(10, 1)

Adding relu

In [ ]:
# Creating sequences
sequence_length = 100

def create_sequences(data, sequence_length):

  total_sequence=[]
  prices = []

  for day in range(0, (len(data) - sequence_length)):
    start_index = day
    end_index = sequence_length + day
    day_sequence = data[start_index:end_index]
    day_price = data[end_index]

    total_sequence.append(day_sequence)
    prices.append(day_price)
  return np.array(total_sequence), np.array(prices)

sequences = create_sequences(scaled_data, sequence_length)
X = sequences[0]
y = sequences[1]

In [ ]:
import torch

# Split the data into training and testing sets
train_size = int(X.shape[0]* split_ratio)
X_train = torch.tensor(X[:train_size]).float()
y_train = torch.tensor(y[:train_size]).float()
X_test = torch.tensor(X[train_size:]).float()
y_test = torch.tensor(y[train_size:]).float()

In [ ]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
      super(MLP, self).__init__()
      self.model = nn.Sequential(
          nn.Linear(input_size, 40),
          nn.ReLU(),
          nn.Linear(40, 20),
          nn.ReLU(),
          nn.Linear(20, 1)
      )

    def forward(self, x):
      x = x.view(-1, input_size)
      x = self.model(x)
      return x

# Instantiate the model
input_size = sequence_length
model = MLP(input_size)

In [ ]:
# Model 4 Parameters

lr = 0.01
momentum= 0.9
epochs = 500

In [ ]:
import torch.optim as optim

criterion_training = nn.L1Loss()
criterion_testing = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
# Putting the y data in the correct format

y_test_actual = torch.tensor(scaler.inverse_transform(y_test))
y_train_actual = torch.tensor(scaler.inverse_transform(y_train))

# Initializing a best model for later use
best_test_error = torch.tensor(float(500))
best_model_state = None

# Train the model
for epoch in range(epochs):

    optimizer.zero_grad()
    model.train()
    pred = model(X_train)

    predictions_train_actual = torch.tensor(scaler.inverse_transform(pred.detach()))
    actual_train_error = criterion_training(predictions_train_actual, y_train_actual)

    loss = criterion_training(pred, y_train)
    # Update model here based on error
    loss.backward()
    optimizer.step()

    model.eval()
    # Evaluate the model on the test data
    with torch.no_grad():
      # Evaluate the model here.
      pred_test = model(X_test)
      loss_test = criterion_testing(pred_test, y_test)

      predictions_test_actual = torch.tensor(scaler.inverse_transform(pred_test.detach()))
      actual_test_error = criterion_testing(predictions_test_actual, y_test_actual)

    #defining a best model
    if actual_test_error < best_test_error:
        best_test_error = actual_test_error
        best_model_state = model.state_dict()

    if epoch %10 == 0:
      print(f"Epoch {epoch}: Training Actual Error= {actual_train_error}, Test Actual Error= {actual_test_error}")

Epoch 0: Training Actual Error= 22.369378874675053, Test Actual Error= 20.779999945224876
Epoch 10: Training Actual Error= 10.354660874129321, Test Actual Error= 9.886283691427657
Epoch 20: Training Actual Error= 4.635412255986947, Test Actual Error= 2.8538468638244887
Epoch 30: Training Actual Error= 3.4021981384067383, Test Actual Error= 2.8290811942056124
Epoch 40: Training Actual Error= 3.4023658270637824, Test Actual Error= 3.8123779873499695
Epoch 50: Training Actual Error= 3.2159951982009156, Test Actual Error= 2.9748699184628196
Epoch 60: Training Actual Error= 3.1361548903229317, Test Actual Error= 3.098536405098057
Epoch 70: Training Actual Error= 3.116827107207867, Test Actual Error= 3.196229464996828
Epoch 80: Training Actual Error= 3.0955141109022306, Test Actual Error= 3.0327859790511282
Epoch 90: Training Actual Error= 3.067172462731226, Test Actual Error= 3.075549531916239
Epoch 100: Training Actual Error= 3.040365540824293, Test Actual Error= 3.064486638767004
Epoch 11

relu is worse because of dead neuron problem

# RNN

# Random Forest

In [ ]:
sea_variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345646 entries, 0 to 345645
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   month                     345646 non-null  int64  
 1   day                       345646 non-null  int64  
 2   timestamp                 345646 non-null  object 
 3   avg_sea_surface_temp      345646 non-null  float64
 4   avg_wind_direction_true   200954 non-null  float64
 5   avg_wind_speed            200622 non-null  float64
 6   avg_visibility            18775 non-null   float64
 7   avg_sea_level_pressure    198507 non-null  float64
 8   avg_air_temperature       204296 non-null  float64
 9   avg_wetbulb_temperature   15669 non-null   float64
 10  avg_dewpoint_temperature  141826 non-null  float64
 11  avg_total_cloud_amount    11877 non-null   float64
dtypes: float64(9), int64(2), object(1)
memory usage: 31.6+ MB


In [ ]:
sea_variables.isna().sum() / len(sea_variables)

month                       0.000000
day                         0.000000
timestamp                   0.000000
avg_sea_surface_temp        0.000000
avg_wind_direction_true     0.418613
avg_wind_speed              0.419574
avg_visibility              0.945681
avg_sea_level_pressure      0.425693
avg_air_temperature         0.408944
avg_wetbulb_temperature     0.954667
avg_dewpoint_temperature    0.589678
avg_total_cloud_amount      0.965638
dtype: float64

In [ ]:
sea_variables = sea_variables.drop(columns=["timestamp"])

In [ ]:
day_agg_df = sea_variables.groupby(["month", "day"]).mean().reset_index()

In [ ]:
day_agg_df.isna().sum() / len(day_agg_df)

month                       0.0
day                         0.0
avg_sea_surface_temp        0.0
avg_wind_direction_true     0.0
avg_wind_speed              0.0
avg_visibility              0.0
avg_sea_level_pressure      0.0
avg_air_temperature         0.0
avg_wetbulb_temperature     0.0
avg_dewpoint_temperature    0.0
avg_total_cloud_amount      0.0
dtype: float64

In [ ]:
day_agg_df = day_agg_df.drop(columns=["month", "day"])

In [ ]:
day_agg_df["avg_sea_surface_temp"] = day_agg_df["avg_sea_surface_temp"].shift(periods=1)

In [ ]:
day_agg_df = day_agg_df.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

X = day_agg_df.drop("avg_sea_surface_temp", axis=1)
y = day_agg_df["avg_sea_surface_temp"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error


std_scaler = StandardScaler()
forest_reg = make_pipeline(std_scaler, RandomForestRegressor(random_state=42))

forest_reg.fit(X_train, y_train)
y_train_predictions = forest_reg.predict(X_train)
forest_mae = mean_absolute_error(y_train, y_train_predictions)

print(f"The training data MAE is {forest_mae} or about {(forest_mae/y_train.mean()*100):.0f}% error")

The training data MAE is 0.11045997393791644 or about 1% error


In [ ]:
y_test_predictions = forest_reg.predict(X_test)
forest_test_mae = mean_absolute_error(y_test, y_test_predictions)

print(f"The test data MAE is {forest_test_mae} or about {(forest_test_mae/y_test.mean()*100):.0f}% error")

The test data MAE is 0.2449696002364195 or about 1% error


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'randomforestregressor__max_depth': randint(low=1, high=50),
                  'randomforestregressor__min_samples_leaf': randint(low=1, high=20)}

rnd_search = RandomizedSearchCV(
    forest_reg, param_distributions=param_distribs, n_iter=50, cv=3,
    scoring='neg_mean_absolute_error', random_state=42)

rnd_search.fit(X_train, y_train)

In [ ]:
rnd_res = pd.DataFrame(rnd_search.cv_results_)
rnd_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
rnd_res.filter(regex = '(^param_|mean_test_score)', axis=1)

,param_randomforestregressor__max_depth,param_randomforestregressor__min_samples_leaf,mean_test_score
6,22,2,-0.337453
15,44,3,-0.341983
5,40,3,-0.341983
4,11,4,-0.345648
9,2,1,-0.346375
8,30,6,-0.351201
19,4,14,-0.351309
14,15,15,-0.351973
0,39,15,-0.351973
13,28,16,-0.353105


In [ ]:
#help hyperparameter tuning made the model worse but the bones are here ig

# Model 1

# Adding more variabes to RNN/MLP

### Bennett: Start running from here

In [17]:
# Creating sequences
sequence_length = 99

def create_sequences(data, sequence_length):

  total_sequence=[]
  prices = []

  for day in range(0, (len(data) - sequence_length)):
    start_index = day
    end_index = sequence_length + day
    day_sequence = data[start_index:end_index]
    day_price = data[end_index]

    total_sequence.append(day_sequence)
    prices.append(day_price)
  return np.array(total_sequence), np.array(prices)

sequences = create_sequences(scaled_data, sequence_length)
X = sequences[0]
y = sequences[1]

In [18]:
import torch

# Split the data into training and testing sets
train_size = int(X.shape[0]* split_ratio)
X_train = torch.tensor(X[:train_size]).float()
y_train = torch.tensor(y[:train_size]).float()
X_test = torch.tensor(X[train_size:]).float()
y_test = torch.tensor(y[train_size:]).float()

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([276437, 99, 1]),
 torch.Size([69110, 99, 1]),
 torch.Size([276437, 1]),
 torch.Size([69110, 1]))

In [15]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

# Example usage:
# Define input parameters
input_size = 1  # Number of features in input data (e.g., time series)
hidden_size = 20  # Number of hidden units in the RNN
num_layers = 2  # Number of RNN layers
output_size = 1  # Number of features in output data (e.g., regression target)

# Instantiate the model
model = RNN(input_size, hidden_size, num_layers, output_size)

# Print model architecture
print(model)

RNN(
  (rnn): RNN(1, 10, num_layers=2, batch_first=True)
  (fc): Linear(in_features=10, out_features=1, bias=True)
)


In [16]:
import torch.optim as optim

# Define the loss function
criterion = nn.L1Loss()  # Mean Absolute Error (MAE) loss

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Calculate MAE loss on the training set
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    outputs = model(X_train)
    mae_loss = criterion(outputs, y_train)
    print(f'MAE Loss on Training Set: {mae_loss.item():.4f}')

Epoch [1/10], Loss: 0.8694
Epoch [2/10], Loss: 0.8577
Epoch [3/10], Loss: 0.8460
Epoch [4/10], Loss: 0.8345
Epoch [5/10], Loss: 0.8230
Epoch [6/10], Loss: 0.8116
Epoch [7/10], Loss: 0.8004
Epoch [8/10], Loss: 0.7892
Epoch [9/10], Loss: 0.7781
Epoch [10/10], Loss: 0.7670
MAE Loss on Training Set: 0.7560


# LSTM

---

In [19]:
# X_train = X_train[0: 20000]
# X_test = X_test[0: 5000]
# y_train = y_train[0: 20000]
# y_test = y_test[0: 5000]

In [20]:
import torch.optim as optim
import torch.nn as nn

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Example usage:
# Define input parameters
input_size = 1  # Number of features in input data (e.g., time series)
hidden_size = 20  # Number of hidden units in the LSTM
num_layers = 2  # Number of LSTM layers
output_size = 1  # Number of features in output data (e.g., regression target)

# Instantiate the model
model = LSTM(input_size, hidden_size, num_layers, output_size)

# Print model architecture
print(model)

LSTM(
  (lstm): LSTM(1, 10, num_layers=2, batch_first=True)
  (fc): Linear(in_features=10, out_features=1, bias=True)
)


In [21]:
# Define the loss function
criterion = nn.L1Loss()  # Mean Absolute Error (MAE) loss

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Calculate MAE loss on the training set
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    outputs = model(X_train)
    mae_loss = criterion(outputs, y_train)
    print(f'MAE Loss on Training Set: {mae_loss.item():.4f}')

Epoch [1/10], Loss: 0.5752
Epoch [2/10], Loss: 0.5680
Epoch [3/10], Loss: 0.5610
Epoch [4/10], Loss: 0.5539
Epoch [5/10], Loss: 0.5469
Epoch [6/10], Loss: 0.5399
Epoch [7/10], Loss: 0.5330
Epoch [8/10], Loss: 0.5260
Epoch [9/10], Loss: 0.5191
Epoch [10/10], Loss: 0.5122
MAE Loss on Training Set: 0.5053
